<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [103]:
from scipy.special import comb

In [104]:
def get_fail_prob( lcomb, nrs, perr ):
    a = []
    for nopen in lcomb:
        p=0.0
        for nsel in np.arange( nopen, nrs, -1 ):
            p+= comb( nopen, nsel) * perr**nsel * (1-perr)**(nopen-nsel)
        a += [p]
    return a


In [105]:
def get_not_pass_prob( lprob ):
    if len(lprob) == 1:
        return lprob[0]
    else:
        recp = get_not_pass_prob( lprob[1:])
        return lprob[0] + recp - lprob[0]*recp


In [106]:
lprob = get_fail_prob( [3,2,1,2,5,6,1], 2, 1e-6)
get_not_pass_prob( lprob )

3.0999940000041991e-17